# Create Enriched Master Dataset

This notebook reads the cleaned yearly CSV files and adds supplemental columns needed for analysis.

**What this does:**
- Reads all `cleaned_*.csv` files from `../data/cleaned/`
- Adds supplemental columns (SEVIS_ID, IS_STEM, geographic mappings, etc.)
- Outputs a single enriched master Parquet file

**Supporting data required:**
- DHS STEM CIP code list (2024)
- CIP code to NSF subject field mapping
- ZIP code to LMA mapping (quarterly, 2010-2024)
- ZIP code to county mapping (quarterly, 2010-2024)
- Working population by county (yearly, 2004-2023)

**Run this once** after cleaning the data and before creating staging tables.

**Expected runtime:** 10-30 minutes depending on your machine and data size.

## Configuration

In [ ]:
# ===== USER CONFIGURATION =====

# Input paths
RAW_DATA_PATH <- '../data/cleaned/cleaned_*_all.csv'
SUPPORTING_DATA_DIR <- '../data/supporting'

# Output path
OUTPUT_PATH <- '../data/sevis_f1_enriched_master.parquet'

# Supporting data files (update these paths as needed)
DHS_STEM_LIST <- file.path(SUPPORTING_DATA_DIR, 'dhs_stem_cip_code_list_July2024.csv')
CIP_TO_NSF_MAPPING <- file.path(SUPPORTING_DATA_DIR, 'cip_code_to_nsf_subject_field_mapping.csv')
ZIP_LMA_MAPPING <- file.path(SUPPORTING_DATA_DIR, 'zip_county_lma_quarterly.csv')
ZIP_COUNTY_CROSSWALK <- file.path(SUPPORTING_DATA_DIR, 'HUD_zip_code_to_county_crosswalk_2010-2024.csv')
WORKING_POP_BY_COUNTY <- file.path(SUPPORTING_DATA_DIR, 'working_pop_by_county_fips_2004-2023.csv')

cat("Configuration:\n")
cat(sprintf("  Input: %s\n", RAW_DATA_PATH))
cat(sprintf("  Output: %s\n", OUTPUT_PATH))
cat(sprintf("  Supporting data: %s\n", SUPPORTING_DATA_DIR))

## Setup

In [ ]:
library(duckdb)
library(DBI)

# Connect to DuckDB
con <- dbConnect(duckdb::duckdb())

cat("✓ DuckDB connection established\n")

# Check that supporting data files exist
required_files <- c(DHS_STEM_LIST, CIP_TO_NSF_MAPPING, ZIP_LMA_MAPPING, 
                    ZIP_COUNTY_CROSSWALK, WORKING_POP_BY_COUNTY)

cat("\nChecking supporting data files:\n")
for (f in required_files) {
  if (file.exists(f)) {
    cat(sprintf("  ✓ %s\n", basename(f)))
  } else {
    cat(sprintf("  ✗ MISSING: %s\n", basename(f)))
    stop(sprintf("Required file not found: %s", f))
  }
}

## Step 1: Create SEVIS_ID

Unique identifier combining Year + Individual_Key

In [ ]:
cat("\n=== Step 1: Creating SEVIS_ID ===")
cat("\nReading raw data and creating SEVIS_ID column...\n")

query_sevis_id <- sprintf("
CREATE OR REPLACE TEMP TABLE base_data AS
SELECT 
  *,
  CONCAT(CAST(Year AS VARCHAR), Individual_Key) AS SEVIS_ID
FROM read_csv_auto('%s', union_by_name=true)
", RAW_DATA_PATH)

dbExecute(con, query_sevis_id)

# Get row count
row_count <- dbGetQuery(con, "SELECT COUNT(*) as count FROM base_data")$count
cat(sprintf("✓ Base data loaded: %s rows\n", format(row_count, big.mark=",")))

## Step 2: Add IS_STEM Column

Marks records as STEM based on DHS STEM CIP code list (considers any major or minor)

In [ ]:
cat("\n=== Step 2: Adding IS_STEM column ===")
cat("\nJoining to DHS STEM CIP code list...\n")

query_is_stem <- sprintf("
CREATE OR REPLACE TEMP TABLE with_stem AS
SELECT 
  b.*,
  CASE
    -- If all CIP fields are missing/blank, keep unknown as NULL
    WHEN NULLIF(TRIM(b.Major_1_CIP_Code), '') IS NULL 
     AND NULLIF(TRIM(b.Major_2_CIP_Code), '') IS NULL 
     AND NULLIF(TRIM(b.Minor_CIP_Code), '') IS NULL
    THEN NULL
    -- Otherwise TRUE if any CIP matches the DHS STEM list
    ELSE EXISTS (
      SELECT 1
      FROM (
        SELECT UNNEST([b.Major_1_CIP_Code, b.Major_2_CIP_Code, b.Minor_CIP_Code]) AS cip
      ) cips
      JOIN read_csv_auto('%s') AS stem_list
        ON LOWER(REPLACE(TRIM(stem_list.\"2020_cip_code\"), ' ', '')) 
         = LOWER(REPLACE(TRIM(cips.cip), ' ', ''))
    )
  END AS IS_STEM
FROM base_data b
", DHS_STEM_LIST)

dbExecute(con, query_is_stem)

# Get STEM count
stem_stats <- dbGetQuery(con, "
  SELECT 
    COUNT(*) as total,
    SUM(CASE WHEN IS_STEM = TRUE THEN 1 ELSE 0 END) as stem_count,
    SUM(CASE WHEN IS_STEM IS NULL THEN 1 ELSE 0 END) as unknown_count
  FROM with_stem
")

cat(sprintf("✓ STEM classification complete:\n"))
cat(sprintf("  - STEM: %s (%.1f%%)\n", 
            format(stem_stats$stem_count, big.mark=","),
            100 * stem_stats$stem_count / stem_stats$total))
cat(sprintf("  - Non-STEM: %s (%.1f%%)\n", 
            format(stem_stats$total - stem_stats$stem_count - stem_stats$unknown_count, big.mark=","),
            100 * (stem_stats$total - stem_stats$stem_count - stem_stats$unknown_count) / stem_stats$total))
cat(sprintf("  - Unknown: %s (%.1f%%)\n", 
            format(stem_stats$unknown_count, big.mark=","),
            100 * stem_stats$unknown_count / stem_stats$total))

## Step 3: Add NSF_SUBJ_FIELD_BROAD Column

Maps CIP codes to NSF broad subject fields (based on first major only)

In [ ]:
cat("\n=== Step 3: Adding NSF_SUBJ_FIELD_BROAD column ===")
cat("\nMapping CIP codes to NSF fields...\n")

# Helper function to normalize CIP codes (MM.mmmm format)
query_nsf <- sprintf("
CREATE OR REPLACE TEMP TABLE with_nsf AS
SELECT 
  w.*,
  m.NSF_BROAD_FIELD AS NSF_SUBJ_FIELD_BROAD
FROM with_stem w
LEFT JOIN (
  SELECT DISTINCT
    -- Normalize CIP code to MM.mmmm format
    CONCAT(
      LPAD(REGEXP_EXTRACT(TRIM(CIPCODE_ORIGINAL), '(\\d+)', 1), 2, '0'),
      '.',
      RPAD(COALESCE(REGEXP_EXTRACT(TRIM(CIPCODE_ORIGINAL), '\\d+\\.(\\d+)$', 1), ''), 4, '0')
    ) AS cip_normalized,
    NSF_BROAD_FIELD
  FROM read_csv_auto('%s')
) m
  ON CONCAT(
       LPAD(REGEXP_EXTRACT(TRIM(CAST(w.Major_1_CIP_Code AS VARCHAR)), '(\\d+)', 1), 2, '0'),
       '.',
       RPAD(COALESCE(REGEXP_EXTRACT(TRIM(CAST(w.Major_1_CIP_Code AS VARCHAR)), '\\d+\\.(\\d+)$', 1), ''), 4, '0')
     ) = m.cip_normalized
", CIP_TO_NSF_MAPPING)

dbExecute(con, query_nsf)

# Get NSF field stats
nsf_stats <- dbGetQuery(con, "
  SELECT 
    COUNT(*) as total,
    COUNT(NSF_SUBJ_FIELD_BROAD) as mapped_count
  FROM with_nsf
")

cat(sprintf("✓ NSF field mapping complete: %s/%s records mapped (%.1f%%)\n",
            format(nsf_stats$mapped_count, big.mark=","),
            format(nsf_stats$total, big.mark=","),
            100 * nsf_stats$mapped_count / nsf_stats$total))

# Show top NSF fields
top_fields <- dbGetQuery(con, "
  SELECT NSF_SUBJ_FIELD_BROAD, COUNT(*) as count
  FROM with_nsf
  WHERE NSF_SUBJ_FIELD_BROAD IS NOT NULL
  GROUP BY NSF_SUBJ_FIELD_BROAD
  ORDER BY count DESC
  LIMIT 5
")

cat("\nTop 5 NSF fields:\n")
for (i in 1:nrow(top_fields)) {
  cat(sprintf("  %d. %s: %s\n", i, top_fields$NSF_SUBJ_FIELD_BROAD[i], 
              format(top_fields$count[i], big.mark=",")))
}

## Step 4: Add Geographic Columns (LMA and County)

Maps ZIP codes to Labor Market Areas (LMAs) and counties based on Program End Date quarter

In [ ]:
cat("\n=== Step 4: Adding geographic columns (CAMPUS_LMA, EMPLOYER_LMA, CAMPUS_COUNTY, EMPLOYER_COUNTY) ===")
cat("\nThis may take several minutes...\n")

# First, create a slim ZIP→LMA lookup table
cat("\nStep 4a: Creating ZIP→LMA lookup...\n")
query_lma_lookup <- sprintf("
CREATE OR REPLACE TEMP TABLE zip_lma_slim AS
WITH cleaned AS (
  SELECT
    CAST(YEAR AS INTEGER) AS year,
    CAST(QUARTER AS INTEGER) AS quarter,
    SUBSTR(REGEXP_REPLACE(TRIM(zip5), '[^0-9]', ''), 1, 5) AS zip5_norm,
    lma_name
  FROM read_csv_auto('%s')
)
SELECT year, quarter, zip5_norm, lma_name
FROM cleaned
WHERE year IS NOT NULL
  AND quarter BETWEEN 1 AND 4
  AND LENGTH(zip5_norm) = 5
  AND zip5_norm <> '00000'
QUALIFY ROW_NUMBER() OVER (
  PARTITION BY year, quarter, zip5_norm
  ORDER BY lma_name
) = 1
", ZIP_LMA_MAPPING)

dbExecute(con, query_lma_lookup)
lma_count <- dbGetQuery(con, "SELECT COUNT(*) as count FROM zip_lma_slim")$count
cat(sprintf("✓ ZIP→LMA lookup created: %s mappings\n", format(lma_count, big.mark=",")))

# Create ZIP→County lookup table
cat("\nStep 4b: Creating ZIP→County lookup...\n")
query_county_lookup <- sprintf("
CREATE OR REPLACE TEMP TABLE zip_county_slim AS
WITH cw AS (
  SELECT
    CAST(YEAR AS INTEGER) AS year,
    CAST(QUARTER AS INTEGER) AS quarter,
    SUBSTR(REGEXP_REPLACE(TRIM(ZIP), '[^0-9]', ''), 1, 5) AS zip5_norm,
    COUNTY AS county5
  FROM read_csv_auto('%s')
),
wp AS (
  SELECT
    LPAD(CAST(CAST(TRIM(CAST(County_LAUS_areacode AS VARCHAR)) AS INTEGER) AS VARCHAR), 5, '0') AS county5,
    County_Name_State_Abbreviation
  FROM read_csv_auto('%s')
)
SELECT
  cw.year,
  cw.quarter,
  cw.zip5_norm,
  wp.County_Name_State_Abbreviation
FROM cw
JOIN wp USING (county5)
WHERE cw.year IS NOT NULL
  AND cw.quarter BETWEEN 1 AND 4
  AND LENGTH(cw.zip5_norm) = 5
  AND cw.zip5_norm <> '00000'
QUALIFY ROW_NUMBER() OVER (
  PARTITION BY year, quarter, zip5_norm
  ORDER BY County_Name_State_Abbreviation
) = 1
", ZIP_COUNTY_CROSSWALK, WORKING_POP_BY_COUNTY)

dbExecute(con, query_county_lookup)
county_count <- dbGetQuery(con, "SELECT COUNT(*) as count FROM zip_county_slim")$count
cat(sprintf("✓ ZIP→County lookup created: %s mappings\n", format(county_count, big.mark=",")))

# Now join to add all 4 geographic columns
cat("\nStep 4c: Joining geographic data to main dataset...\n")
query_geo <- "
CREATE OR REPLACE TEMP TABLE with_geo AS
SELECT 
  n.*,
  lma_campus.lma_name AS CAMPUS_LMA,
  lma_employer.lma_name AS EMPLOYER_LMA,
  county_campus.County_Name_State_Abbreviation AS CAMPUS_COUNTY,
  county_employer.County_Name_State_Abbreviation AS EMPLOYER_COUNTY
FROM with_nsf n
LEFT JOIN zip_lma_slim lma_campus
  ON EXTRACT(YEAR FROM TRY_CAST(n.Program_End_Date AS DATE)) = lma_campus.year
 AND EXTRACT(QUARTER FROM TRY_CAST(n.Program_End_Date AS DATE)) = lma_campus.quarter
 AND n.Campus_Zip_Code = lma_campus.zip5_norm
LEFT JOIN zip_lma_slim lma_employer
  ON EXTRACT(YEAR FROM TRY_CAST(n.Program_End_Date AS DATE)) = lma_employer.year
 AND EXTRACT(QUARTER FROM TRY_CAST(n.Program_End_Date AS DATE)) = lma_employer.quarter
 AND n.Employer_Zip_Code = lma_employer.zip5_norm
LEFT JOIN zip_county_slim county_campus
  ON EXTRACT(YEAR FROM TRY_CAST(n.Program_End_Date AS DATE)) = county_campus.year
 AND EXTRACT(QUARTER FROM TRY_CAST(n.Program_End_Date AS DATE)) = county_campus.quarter
 AND n.Campus_Zip_Code = county_campus.zip5_norm
LEFT JOIN zip_county_slim county_employer
  ON EXTRACT(YEAR FROM TRY_CAST(n.Program_End_Date AS DATE)) = county_employer.year
 AND EXTRACT(QUARTER FROM TRY_CAST(n.Program_End_Date AS DATE)) = county_employer.quarter
 AND n.Employer_Zip_Code = county_employer.zip5_norm
"

dbExecute(con, query_geo)

# Get match rates
geo_stats <- dbGetQuery(con, "
  SELECT 
    COUNT(*) as total,
    COUNT(CAMPUS_LMA) as campus_lma_count,
    COUNT(EMPLOYER_LMA) as employer_lma_count,
    COUNT(CAMPUS_COUNTY) as campus_county_count,
    COUNT(EMPLOYER_COUNTY) as employer_county_count
  FROM with_geo
")

cat("\n✓ Geographic columns added:\n")
cat(sprintf("  - CAMPUS_LMA: %s/%s (%.1f%%)\n",
            format(geo_stats$campus_lma_count, big.mark=","),
            format(geo_stats$total, big.mark=","),
            100 * geo_stats$campus_lma_count / geo_stats$total))
cat(sprintf("  - EMPLOYER_LMA: %s/%s (%.1f%%)\n",
            format(geo_stats$employer_lma_count, big.mark=","),
            format(geo_stats$total, big.mark=","),
            100 * geo_stats$employer_lma_count / geo_stats$total))
cat(sprintf("  - CAMPUS_COUNTY: %s/%s (%.1f%%)\n",
            format(geo_stats$campus_county_count, big.mark=","),
            format(geo_stats$total, big.mark=","),
            100 * geo_stats$campus_county_count / geo_stats$total))
cat(sprintf("  - EMPLOYER_COUNTY: %s/%s (%.1f%%)\n",
            format(geo_stats$employer_county_count, big.mark=","),
            format(geo_stats$total, big.mark=","),
            100 * geo_stats$employer_county_count / geo_stats$total))

## Step 5: Add Working Population Columns

Adds working population data for employer LMA and state (for context/normalization)

In [ ]:
cat("\n=== Step 5: Adding working population columns ===")
cat("\nUnpivoting working population data...\n")

# Create unpivoted working population lookup
query_workpop <- sprintf("
CREATE OR REPLACE TEMP TABLE workpop_long AS
SELECT
  County_Name_State_Abbreviation,
  state_name,
  CAST(year_col AS INTEGER) AS year,
  CASE
    WHEN LOWER(TRIM(pop_value)) IN ('missing data', '') THEN NULL
    ELSE CAST(REPLACE(TRIM(pop_value), ',', '') AS INTEGER)
  END AS working_pop
FROM (
  SELECT
    County_Name_State_Abbreviation,
    state_name,
    UNNEST([
      '2004','2005','2006','2007','2008','2009','2010','2011','2012','2013',
      '2014','2015','2016','2017','2018','2019','2020','2021','2022','2023','2024'
    ]) AS year_col,
    UNNEST([
      LMA_WORKING_POP_2004, LMA_WORKING_POP_2005, LMA_WORKING_POP_2006, LMA_WORKING_POP_2007,
      LMA_WORKING_POP_2008, LMA_WORKING_POP_2009, LMA_WORKING_POP_2010, LMA_WORKING_POP_2011,
      LMA_WORKING_POP_2012, LMA_WORKING_POP_2013, LMA_WORKING_POP_2014, LMA_WORKING_POP_2015,
      LMA_WORKING_POP_2016, LMA_WORKING_POP_2017, LMA_WORKING_POP_2018, LMA_WORKING_POP_2019,
      LMA_WORKING_POP_2020, LMA_WORKING_POP_2021, LMA_WORKING_POP_2022, LMA_WORKING_POP_2023,
      LMA_WORKING_POP_2024
    ]) AS pop_value
  FROM read_csv_auto('%s')
)
WHERE year BETWEEN 2004 AND 2024
", WORKING_POP_BY_COUNTY)

dbExecute(con, query_workpop)

# Aggregate to state level
cat("Creating state-level aggregates...\n")
dbExecute(con, "
CREATE OR REPLACE TEMP TABLE workpop_state AS
SELECT
  state_name,
  year,
  SUM(working_pop) AS state_working_pop
FROM workpop_long
WHERE state_name IS NOT NULL
GROUP BY state_name, year
")

# Join to main dataset
cat("Joining working population data...\n")
query_final <- "
CREATE OR REPLACE TEMP TABLE enriched_master AS
SELECT 
  g.*,
  wp_lma.working_pop AS EMPLOYER_LMA_WORKPOP_YR,
  wp_state.state_working_pop AS EMPLOYER_STATE_WORKPOP_YR
FROM with_geo g
LEFT JOIN workpop_long wp_lma
  ON g.EMPLOYER_COUNTY = wp_lma.County_Name_State_Abbreviation
 AND EXTRACT(YEAR FROM TRY_CAST(g.Authorization_Start_Date AS DATE)) = wp_lma.year
LEFT JOIN workpop_state wp_state
  ON g.Employer_State = wp_state.state_name
 AND EXTRACT(YEAR FROM TRY_CAST(g.Authorization_Start_Date AS DATE)) = wp_state.year
"

dbExecute(con, query_final)

workpop_stats <- dbGetQuery(con, "
  SELECT 
    COUNT(*) as total,
    COUNT(EMPLOYER_LMA_WORKPOP_YR) as lma_pop_count,
    COUNT(EMPLOYER_STATE_WORKPOP_YR) as state_pop_count
  FROM enriched_master
")

cat("\n✓ Working population columns added:\n")
cat(sprintf("  - EMPLOYER_LMA_WORKPOP_YR: %s/%s (%.1f%%)\n",
            format(workpop_stats$lma_pop_count, big.mark=","),
            format(workpop_stats$total, big.mark=","),
            100 * workpop_stats$lma_pop_count / workpop_stats$total))
cat(sprintf("  - EMPLOYER_STATE_WORKPOP_YR: %s/%s (%.1f%%)\n",
            format(workpop_stats$state_pop_count, big.mark=","),
            format(workpop_stats$total, big.mark=","),
            100 * workpop_stats$state_pop_count / workpop_stats$total))

## Step 6: Export Enriched Master Dataset

Write the final enriched dataset to a compressed Parquet file

In [ ]:
cat("\n=== Step 6: Exporting enriched master dataset ===")
cat("\nWriting to Parquet (this may take several minutes)...\n")

start_time <- Sys.time()

query_export <- sprintf("
COPY (
  SELECT * FROM enriched_master
) TO '%s' (FORMAT PARQUET, COMPRESSION ZSTD, ROW_GROUP_SIZE 100000)
", OUTPUT_PATH)

dbExecute(con, query_export)

elapsed_time <- as.numeric(difftime(Sys.time(), start_time, units="secs"))
file_size <- file.info(OUTPUT_PATH)$size

format_file_size <- function(size_bytes) {
  units <- c('B', 'KB', 'MB', 'GB', 'TB')
  unit_index <- 1
  size <- size_bytes
  
  while (size >= 1024 && unit_index < length(units)) {
    size <- size / 1024
    unit_index <- unit_index + 1
  }
  
  sprintf("%.1f %s", size, units[unit_index])
}

cat("\n✓ Enriched master dataset created!\n")
cat(sprintf("  - Output: %s\n", OUTPUT_PATH))
cat(sprintf("  - Size: %s\n", format_file_size(file_size)))
cat(sprintf("  - Time: %.1f seconds\n", elapsed_time))
cat(sprintf("  - Rows: %s\n", format(workpop_stats$total, big.mark=",")))

## Summary and Cleanup

In [ ]:
cat("\n" %+% paste(rep("=", 80), collapse="") %+% "\n")
cat("ENRICHED MASTER DATASET CREATION COMPLETE\n")
cat(paste(rep("=", 80), collapse="") %+% "\n\n")

cat("Added columns:\n")
cat("  1. SEVIS_ID - Unique identifier (Year + Individual_Key)\n")
cat("  2. IS_STEM - STEM classification based on DHS list\n")
cat("  3. NSF_SUBJ_FIELD_BROAD - NSF broad subject field\n")
cat("  4. CAMPUS_LMA - Campus labor market area\n")
cat("  5. EMPLOYER_LMA - Employer labor market area\n")
cat("  6. CAMPUS_COUNTY - Campus county name\n")
cat("  7. EMPLOYER_COUNTY - Employer county name\n")
cat("  8. EMPLOYER_LMA_WORKPOP_YR - Working population in employer LMA\n")
cat("  9. EMPLOYER_STATE_WORKPOP_YR - Working population in employer state\n")

cat("\n✓ You can now run create_staging_tables.ipynb to create analysis-ready staging tables.\n")

# Disconnect
dbDisconnect(con, shutdown = TRUE)
cat("\n✓ Database connection closed\n")

## Next Steps

1. **Verify the output**: Check that `sevis_f1_enriched_master.parquet` was created successfully
2. **Update staging tables notebook**: The `create_staging_tables.ipynb` notebook will be updated to read from this enriched master file
3. **Run staging tables**: Execute `create_staging_tables.ipynb` to create analysis-ready staging tables
4. **Run analyses**: Use the individual analysis notebooks to explore the data